# Modeling rain in a cloudy column
In this notebook, you use a column model to study cloud microphysical processes that are important for rain. The notebook consists of two parts. Part 1 is a tutorial that introduces you to the model and its usage. In part 2 you will independently explore the microphysics of rain in some sensitivity studies.

# Part 1: The column model

The column model simulates the evolution of temperature, moisture and rain in a cloudy column of the boundary layer. This means that the atmospheric fields vary with the height in the column but do not vary spatially, i.e. the model simulates the evolution of profiles. The model starts from an existing cloud and simulates the conversion of this cloud, which will eventually cease to exist, into rain. The following processes are considered:

* formation of rain drops by collision-coalescence of cloud droplets (autoconversion, auto)
* growth of rain drops by collision-coalescence with cloud droplets (accretion, accr)
* sedimentation of rain drops (sedi)
* evaporation of rain drops below the cloud (evap)

A summary of the model, including its governing equations, is provided below.

---

**Geometry**: column/profiles

**Dimensions**: height z, time t

**Prognostic (simulated) variables**: 
    
   - liquid water potential temperature &theta;<sub>L</sub>(t,z); *variable name: 'thl'*
   - total non-rain water mixing ratio q<sub>T</sub>(t,z) (including water vapor and cloud droplets but excluding rain drops); *variable name: 'qt'*
   - rain drop water mixing ratio q<sub>R</sub>(t,z); *variable name: 'qr'*   

**Diagnostic (derived) variable**: 

   - cloud droplet water mixing ratio q<sub>L</sub>(t,z); *variable name: 'ql'*

**Equations**:

   - &part;<sub>t</sub> &theta;<sub>L</sub>(t,z) = - auto - accr + evap
   - &part;<sub>t</sub> q<sub>T</sub>(t,z) = - auto - accr + evap
   - &part;<sub>t</sub> q<sub>R</sub>(t,z) = + auto + accr - evap &pm; sedi
   - q<sub>L</sub>(t,z) = max(q<sub>T</sub>(t,z) - q<sub>S</sub>(t,z), 0), where q<sub>S</sub> is the saturation water vapor mixing ratio.

**Representation of processes**:

   - auto = c q<sub>L</sub><sup>&alpha;</sup> N<sup>-&beta;</sup>
   - accr = d q<sub>L</sub><sup>&gamma;</sup> q<sub>R</sub><sup>&delta;</sup>
   
   where c, d, &alpha;, &beta;, &gamma;, &delta; > 0 are positive constants and N is the number concentration of cloud droplets (*variable name 'nc'*), which we assume to have a constant value. We focus here on autoconversion and accretion. Details on sedimentation and below-cloud evaporation of rain can be found in Khairoutdinov and Kogan (2000).

---

The code cell below initializes and runs the column model. You can execute code cells by first selecting them with the mouse and then pressing Shift+Enter. It may take a few seconds for the model run to finish.

In [ ]:
# Import/load the single column and some routines for plotting
from run_micro import run_column_model
from src.plot_util import plot_initial_profiles, plot_profile_evolution, plot_timeseries

# run the model with specified settings
ref_run = run_column_model(
              run_name='ref',
              thl=298,      # liquid water potential temperature, unit: K
              qt=15e-3,     # total water mixing ratio, unit: kg/kg 
              nc=80e6,      # cloud droplet number concentration, unit: 1/m3
              sw_auto=True, # autoconverion: on
              sw_accr=True, # accretion: on
              sw_evap=True, # rain drop evaporation: on
              sw_sedi=True, # rain drop sedimentation: on
         )

## 1. The initial cloud
The column model starts from an existing cloud. The following cell plots this initial condition using different conserved variables.

In [ ]:
plot_initial_profiles(ref_run)

### Exercise 1.1

- Identify the location of cloud base and cloud top in the profiles.
- Explain the differences between the potential and liquid water potential temperature and between the total water and water vapor mixing ratio.
- What type of cloud is simulated?

## 2. Rain formation

The column model simulates what happens to the initial cloud. The following cell plots how the cloud as represented by q<sub>L</sub> evolves over time.

In [ ]:
plot_profile_evolution([ref_run], 'ql')

### Exercise 1.2

1. Plot the evolution of the profile of q<sub>R</sub> (*variable name 'qr'*).

*Hint: Use Insert->Insert Cell Below to create a new cell for the required code.*

2. Explain, why...
    - ... q<sub>L</sub> decreases over time.
    - ... the shape of the q<sub>R</sub> profile changes over time.

The processes of autoconversion and accretion result in rain drops that sediment towards the ground as rainfall. The next code box plots the surface rain rate as function of time, i.e., the time series of the rain rate. The value in the legend indicates the total rainfall after 60 minutes

In [ ]:
plot_timeseries([ref_run], 'surf_rain_rate')

# Part 2: Sensitivity studies

You are now equipped to investigate how the rain rate depends on the processes of accretion and autoconversion as well as on the conditions in the well-mixed sub-cloud layer. We describe the conditions in the sub-cloud layer by the mixed-layer values for &theta;<sub>L</sub> and q<sub>t</sub>. We also investigate the effect of different aerosol particle concentrations in the mixed-layer by changing the cloud droplet number concentration N (recall from the lecture that the cloud droplet number concentration depends on the atmospheric aerosol conditions).

A. Investigate the sensitivity of rainfall to these factors by running and comparing the following simulations:

  1. The reference run (You already ran this simulation, recall how that worked.)
  2. A run where you disable accretion by setting sw_accr=False.
  3. A run where you disable autoconversion by setting sw_auto=False.
  4. A run where you increase q<sub>t</sub> by 1 &permil; (qt = 1.001 * 15e-3 kg kg-1).
  5. A run where you increase &theta;<sub>L</sub> by 1 &permil; (thl = 1.001 * 298 K).
  6. A run where you increase the cloud droplet number concentration by 10 % (nc = 1.1 * 80e6 m-3).

Try to explain the observed sensitivities based on the model equations.

*Hint: You can plot different runs in the same plot by using [ref_run, run2] instead of [ref_run] in the plotting routines.*

B. Comment on the following statements:

* "Accretion is much more important for rain formation than autoconversion."
* "Rain is equally sensitive to temperature, moisture and the aerosol conditions."